# Contents
- Download data
- Create full_data columns
- Create train and datasets
- Train models
    - Multinomial Naive Bayesian
    - Logistic Regression and SVM
    - Transform

In [2]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from gensim.models.doc2vec import Doc2Vec,\
    TaggedDocument


In [1]:
from nltk.tokenize import word_tokenize

In [3]:
# If is_train is False, only DistilBERT model will not be trained
is_train = False

### Get the data

In [4]:
reddit_combi_df = pd.read_csv(
    "cleaned_data/Reddit_Combi_cleaned.csv",   
    )

reddit_title_df = pd.read_csv(    
    "cleaned_data/Reddit_Title_cleaned.csv",    
)

twitter_full_df = pd.read_csv(
    "cleaned_data/Twitter_Full_cleaned.csv",  
    )

twitter_non_advert = pd.read_csv(
    "cleaned_data/Twitter_Non-Advert_cleaned.csv",
    )

df_files = [reddit_combi_df, reddit_title_df, twitter_full_df, twitter_non_advert]

In [5]:
for df in df_files:
    print(df.columns)

Index(['title', 'body', 'Body_Title', 'label'], dtype='object')
Index(['title', 'label'], dtype='object')
Index(['text', 'hashtags', 'label'], dtype='object')
Index(['text', 'label'], dtype='object')


In [6]:
# Rename column name for consistency
df_files[0] = df_files[0].rename(columns={"Body_Title": "body_title"})
df_files[0].columns

Index(['title', 'body', 'body_title', 'label'], dtype='object')

In [7]:
# merge columns in dataframed containing text
df_files[0]["full_text"] = (
    df_files[0]["title"].fillna("") + " " +
    df_files[0]["body"].fillna("") + " " +
    df_files[0]["body_title"].fillna("") + " " 
)

df_files[1]["full_text"] = (
    df_files[1]["title"].fillna("") + " "
)

df_files[2]["full_text"] = (
    df_files[2]["text"].fillna("") + " " +
    df_files[2]["hashtags"].astype(str).fillna("")
)

df_files[3]["full_text"] = (
    df_files[3]["text"].fillna("") + " " 
)


In [8]:
# clean hashtag symbols from full text, hashtags should only be in Twitter full, but clean from all to maintain consistency
def clean_text(text):
    text = str(text) 
    text = re.sub(r"[\[\]']", "", text)  
    text = text.replace("#", "") 
    return text

for df in df_files:
    df["full_text"] = df["full_text"].apply(clean_text)

In [9]:
for df in df_files:
    print(df.columns)

Index(['title', 'body', 'body_title', 'label', 'full_text'], dtype='object')
Index(['title', 'label', 'full_text'], dtype='object')
Index(['text', 'hashtags', 'label', 'full_text'], dtype='object')
Index(['text', 'label', 'full_text'], dtype='object')


In [10]:
# check full_text is ok

df_files[0].head()

,title,body,body_title,label,full_text
0,envy to other is swallowing me,"im from developingcountry, indonesia , and for...",envy to other is swallowing me im from develop...,1,envy to other is swallowing me im from develop...
1,nothin outta the ordinary. paradise. job stres...,um hello .well many can relate im sure. after ...,nothin outta the ordinary. paradise. job stres...,1,nothin outta the ordinary. paradise. job stres...
2,almost 49 and the chasm of emptiness has never...,i’ve been diagnosed severe bi polar where you ...,almost 49 and the chasm of emptiness has never...,1,almost 49 and the chasm of emptiness has never...
3,i’m happy again,"after my closest friend left me in april, i ha...",i’m happy again after my closest friend left m...,0,i’m happy again after my closest friend left m...
4,is it possible to recover from such a traumati...,"i am only 15, and yet i feel my life is alread...",is it possible to recover from such a traumati...,1,is it possible to recover from such a traumati...


### Clean dataframes, so they contain only one feature column "full_text"

In [11]:
# create new dataframed containing only full_text and label for all dataframes

reddit1 = df_files[0].copy()
reddit2 = df_files[1].copy()
twitter1 = df_files[2].copy()
twitter2 = df_files[3].copy()

reddit1 = reddit1.drop(columns=['title', 'body', 'body_title'])
reddit2 = reddit2.drop(columns=['title'])
twitter1 = twitter1.drop(columns=['text', 'hashtags'])
twitter2 = twitter2.drop(columns=['text'])

df_files_new = [reddit1, reddit2, twitter1, twitter2]


In [12]:
for df in df_files_new:
    print(len(df))
    print(df.columns)

3123
Index(['label', 'full_text'], dtype='object')
5480
Index(['label', 'full_text'], dtype='object')
8525
Index(['label', 'full_text'], dtype='object')
1972
Index(['label', 'full_text'], dtype='object')


In [13]:
df_files_new[0].head()

,label,full_text
0,1,envy to other is swallowing me im from develop...
1,1,nothin outta the ordinary. paradise. job stres...
2,1,almost 49 and the chasm of emptiness has never...
3,0,i’m happy again after my closest friend left m...
4,1,is it possible to recover from such a traumati...


In [26]:
# Create dataframe to summarize accuracies
accuracy_df = pd.DataFrame(columns=[
    "dataset_split",   
    "model",          
    "representation",  
    "accuracy"         
])

## Training data

- Three models are trained and data splits are the following
    - training: reddit, test and validation: twitter
        - how different datasets generalize
    - training: twitter, test and validation: reddit
        - how different datasets generalize
    - training: 80% of all datasets, test and validation: 20% of all datasets
        - typical ML data split

In [15]:
# Split data as above

X_train1 = pd.concat([reddit1, reddit2])["full_text"]
y_train1 = pd.concat([reddit1, reddit2])["label"]
X_test1 = pd.concat([twitter1, twitter2])["full_text"]
y_test1 = pd.concat([twitter1, twitter2])["label"]

X_train2 = pd.concat([twitter1, twitter2])["full_text"]
y_train2 = pd.concat([twitter1, twitter2])["label"]
X_test2 = pd.concat([reddit1, reddit2])["full_text"]
y_test2 = pd.concat([reddit1, reddit2])["label"]

# Merge dataframes
X_3 = pd.concat([reddit1, reddit2, twitter1, twitter2])["full_text"]
y_3 = pd.concat([reddit1, reddit2, twitter1, twitter2])["label"]

X_train3, X_test3, y_train3, y_test3 = train_test_split(X_3, y_3, test_size=0.2, random_state=42)

In [16]:
# check some lengths
print(len(X_train1))
print(len(X_test1))
print(len(X_train3))

8603
10497
15280


## Multinomial Naive Bayesian

- Is selected as a model, because it's widely used in text classification tasks (https://towardsdatascience.com/multinomial-naive-bayes-for-documents-classification-and-natural-language-processing-nlp-e08cc848ce6/)


In [17]:
# Pipeline for embedding + training
# TfidfVectorizer collects a document of TF-IDF features, used in text classification: https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html#sphx-glr-auto-examples-text-plot-document-classification-20newsgroups-py

pip = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words='english',       # remove short, meaningless words
        ngram_range=(1,2)        # to make dict smaller
    )),
    ('nb', MultinomialNB())
])

In [18]:
# train: reddit, test: twitter
pip.fit(X_train1, y_train1)
y_pred1 = pip.predict(X_test1)
acc_MNB_1 = accuracy_score(y_test1, y_pred1)
print("Accuracy:", acc_MNB_1)
print(classification_report(y_test1, y_pred1))


Accuracy: 0.6674287891778603
              precision    recall  f1-score   support

           0       0.87      0.34      0.49      4924
           1       0.62      0.95      0.75      5573

    accuracy                           0.67     10497
   macro avg       0.75      0.65      0.62     10497
weighted avg       0.74      0.67      0.63     10497



In [27]:
accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train: Reddit / test: Twitter",
        "model": "MultinomialNB",
        "representation": "TF-IDF",
        "accuracy": acc_MNB_1
    }])
], ignore_index=True)

C:\Users\oonas\AppData\Local\Temp\ipykernel_32892\3958619609.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  accuracy_df = pd.concat([


In [20]:
# train: twitter, test: reddit
pip.fit(X_train2, y_train2)
y_pred2 = pip.predict(X_test2)
acc_MNB_2 = accuracy_score(y_test2, y_pred2)
print("Accuracy:", acc_MNB_2)
print(classification_report(y_test2, y_pred2))



Accuracy: 0.8000697431128676
              precision    recall  f1-score   support

           0       0.88      0.53      0.66      3189
           1       0.78      0.96      0.86      5414

    accuracy                           0.80      8603
   macro avg       0.83      0.75      0.76      8603
weighted avg       0.81      0.80      0.79      8603



In [28]:
accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train: Twitter / test: Reddit",
        "model": "MultinomialNB",
        "representation": "TF-IDF",
        "accuracy": acc_MNB_2
    }])
], ignore_index=True)


In [22]:
# train: 80%, test: 20%
pip.fit(X_train3, y_train3)
y_pred3 = pip.predict(X_test3)
acc_MNB_3 = accuracy_score(y_test3, y_pred3)
print("Accuracy:", acc_MNB_3)
print(classification_report(y_test3, y_pred3))


Accuracy: 0.8109947643979057
              precision    recall  f1-score   support

           0       0.91      0.62      0.74      1630
           1       0.77      0.96      0.85      2190

    accuracy                           0.81      3820
   macro avg       0.84      0.79      0.79      3820
weighted avg       0.83      0.81      0.80      3820



In [29]:
accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train/test 80/20 split",
        "model": "MultinomialNB",
        "representation": "TF-IDF",
        "accuracy": acc_MNB_3
    }])
], ignore_index=True)


In [30]:
accuracy_df

,dataset_split,model,representation,accuracy
0,train: Reddit / test: Twitter,MultinomialNB,TF-IDF,0.667429
1,train: Twitter / test: Reddit,MultinomialNB,TF-IDF,0.800070
2,train/test 80/20 split,MultinomialNB,TF-IDF,0.810995


- Third dataset split peformed the best, receiving highest accuracy and f1-score (or tie with second model)
- Twitter data as a dataset performed significantly better than reddit data, suggesting the quality of twitter data is better
    - Twitter data generalizes better to new observations
    - Twitter data itself is almost as good in quality than 80% of all datasets, suggesting twitter data could be used as it's own in model training


# SVM and Logstic Regression

### Compare models and two embeddings: TF-IDF and Doc2Vec
- TF-IDF creates matrix of tf-idf features (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
- Doc2Vec learns embeddings of words, numerical vectors (https://www.geeksforgeeks.org/nlp/doc2vec-in-nlp/)

### Training and testing data
- Use same data splits as in NBC

In [31]:
# TF-IDF + Logistic regression
TF_IDF_log_pip = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words='english',       # remove short, meaningless words
        ngram_range=(1,2)        # to make dict smaller
    )),
    ('nb', LogisticRegression())
])

In [32]:
# TF-IDF + SVC
TF_IDF_svm_pip = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words='english',       # remove short, meaningless words
        ngram_range=(1,2)        # to make dict smaller
    )),
    ('nb', svm.SVC())
])

### Doc2Vec processing 
- Vectorize words only once, because it takes over one minute

In [38]:
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")


[nltk_data] Downloading package punkt to c:\Users\oonas\OneDrive\Työpö
[nltk_data]     ytä\DHHB\project\.venv\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to c:\Users\oonas\OneDrive\T
[nltk_data]     yöpöytä\DHHB\project\.venv\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [39]:
def doc2vec(X_train, X_test):
    tagged_train = [TaggedDocument(words=word_tokenize(doc.lower()), tags=[str(i)]) for i, doc in enumerate(X_train)]

    # train Doc2Vev
    model = Doc2Vec(vector_size=20, min_count=2, epochs=50)
    model.build_vocab(tagged_train)
    model.train(tagged_train, total_examples=model.corpus_count, epochs=model.epochs)

    # get document vectors
    train_vec = [model.dv[str(i)] for i in range(len(tagged_train))]
    test_vec = [model.infer_vector(word_tokenize(doc.lower())) for doc in X_test]

    return np.array(train_vec), np.array(test_vec)

In [40]:
X_train_1_d2v, X_test_1_d2v = doc2vec(X_train1.astype(str).tolist(), X_test1.astype(str).tolist())

In [41]:
X_train_2_d2v, X_test_2_d2v = doc2vec(X_train2.astype(str).tolist(), X_test2.astype(str).tolist())

In [42]:
X_train_3_d2v, X_test_3_d2v = doc2vec(X_train3.astype(str).tolist(), X_test3.astype(str).tolist())

### TF-IDF tokenizer

In [43]:
# train: reddit, test: twitter
TF_IDF_log_pip.fit(X_train1, y_train1)
y_pred1_1 = TF_IDF_log_pip.predict(X_test1)

TF_IDF_svm_pip.fit(X_train1, y_train1)
y_pred1_2 = TF_IDF_svm_pip.predict(X_test1)

In [44]:
# Logistic Regression
acc_lr = accuracy_score(y_test1, y_pred1_1)
print("Logistic Regression")
print("Accuracy:", acc_lr)
print(classification_report(y_test1, y_pred1_1))

accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train: Reddit / test: Twitter",
        "model": "Logistic Regression",
        "representation": "TF-IDF",
        "accuracy": acc_lr
    }])
], ignore_index=True)

# SVM
acc_svc = accuracy_score(y_test1, y_pred1_2)
print("SVM")
print("Accuracy:", acc_svc)
print(classification_report(y_test1, y_pred1_2))

accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train: Reddit / test: Twitter",
        "model": "SVM",
        "representation": "TF-IDF",
        "accuracy": acc_svc
    }])
], ignore_index=True)


Logistic Regression
Accuracy: 0.7370677336381823
              precision    recall  f1-score   support

           0       0.74      0.69      0.71      4924
           1       0.74      0.78      0.76      5573

    accuracy                           0.74     10497
   macro avg       0.74      0.73      0.73     10497
weighted avg       0.74      0.74      0.74     10497

SVM
Accuracy: 0.7304944269791369
              precision    recall  f1-score   support

           0       0.71      0.71      0.71      4924
           1       0.75      0.75      0.75      5573

    accuracy                           0.73     10497
   macro avg       0.73      0.73      0.73     10497
weighted avg       0.73      0.73      0.73     10497



In [45]:
# train: twitter, test: reddit
TF_IDF_log_pip.fit(X_train2, y_train2)
y_pred2_1 = TF_IDF_log_pip.predict(X_test2)

TF_IDF_svm_pip.fit(X_train2, y_train2)
y_pred2_2 = TF_IDF_svm_pip.predict(X_test2)

In [46]:
# Logistic Regression
acc_lr2 = accuracy_score(y_test2, y_pred2_1)
print("Logistic Regression")
print("Accuracy:", acc_lr2)
print(classification_report(y_test2, y_pred2_1))

accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train: Twitter / test: Reddit",
        "model": "Logistic Regression",
        "representation": "TF-IDF",
        "accuracy": acc_lr2
    }])
], ignore_index=True)

# SVM
acc_svc2 = accuracy_score(y_test2, y_pred2_2)
print("SVM")
print("Accuracy:", acc_svc2)
print(classification_report(y_test2, y_pred2_2))

accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train: Twitter / test: Reddit",
        "model": "SVM",
        "representation": "TF-IDF",
        "accuracy": acc_svc2
    }])
], ignore_index=True)

Logistic Regression
Accuracy: 0.7927467162617692
              precision    recall  f1-score   support

           0       0.73      0.71      0.72      3189
           1       0.83      0.84      0.84      5414

    accuracy                           0.79      8603
   macro avg       0.78      0.77      0.78      8603
weighted avg       0.79      0.79      0.79      8603

SVM
Accuracy: 0.8029757061490178
              precision    recall  f1-score   support

           0       0.79      0.64      0.71      3189
           1       0.81      0.90      0.85      5414

    accuracy                           0.80      8603
   macro avg       0.80      0.77      0.78      8603
weighted avg       0.80      0.80      0.80      8603



In [47]:
# train: 80&, test: 20%
TF_IDF_log_pip.fit(X_train3, y_train3)
y_pred3_1 = TF_IDF_log_pip.predict(X_test3)

TF_IDF_svm_pip.fit(X_train3, y_train3)
y_pred3_2 = TF_IDF_svm_pip.predict(X_test3)

In [48]:
# Logistic Regression
acc_lr3 = accuracy_score(y_test3, y_pred3_1)
print("Logistic Regression")
print("Accuracy:", acc_lr3)
print(classification_report(y_test3, y_pred3_1))

accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train/test 80/20 split",
        "model": "Logistic Regression",
        "representation": "TF-IDF",
        "accuracy": acc_lr3
    }])
], ignore_index=True)

# SVM
acc_svc3 = accuracy_score(y_test3, y_pred3_2)
print("SVM")
print("Accuracy:", acc_svc3)
print(classification_report(y_test3, y_pred3_2))

accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train/test 80/20 split",
        "model": "SVM",
        "representation": "TF-IDF",
        "accuracy": acc_svc3
    }])
], ignore_index=True)


Logistic Regression
Accuracy: 0.8667539267015707
              precision    recall  f1-score   support

           0       0.88      0.79      0.84      1630
           1       0.86      0.92      0.89      2190

    accuracy                           0.87      3820
   macro avg       0.87      0.86      0.86      3820
weighted avg       0.87      0.87      0.87      3820

SVM
Accuracy: 0.881413612565445
              precision    recall  f1-score   support

           0       0.89      0.82      0.86      1630
           1       0.88      0.92      0.90      2190

    accuracy                           0.88      3820
   macro avg       0.88      0.87      0.88      3820
weighted avg       0.88      0.88      0.88      3820



In [49]:
accuracy_df

,dataset_split,model,representation,accuracy
0,train: Reddit / test: Twitter,MultinomialNB,TF-IDF,0.667429
1,train: Twitter / test: Reddit,MultinomialNB,TF-IDF,0.800070
2,train/test 80/20 split,MultinomialNB,TF-IDF,0.810995
3,train: Reddit / test: Twitter,Logistic Regression,TF-IDF,0.737068
4,train: Reddit / test: Twitter,SVM,TF-IDF,0.730494
5,train: Twitter / test: Reddit,Logistic Regression,TF-IDF,0.792747
6,train: Twitter / test: Reddit,SVM,TF-IDF,0.802976
7,train/test 80/20 split,Logistic Regression,TF-IDF,0.866754
8,train/test 80/20 split,SVM,TF-IDF,0.881414


## Doc2Vec Tokenizer

In [50]:
log_reg_model = LogisticRegression()
SVC_model = svm.SVC()

log_reg_model_2 = LogisticRegression()
SVC_model_2 = svm.SVC()

log_reg_model_3 = LogisticRegression()
SVC_model_3 = svm.SVC()

### Training data: Reddit, testing data: Twitter

In [51]:
log_reg_model.fit(X_train_1_d2v, y_train1)
y_pred1_lr_d2v = log_reg_model.predict(X_test_1_d2v) 

SVC_model.fit(X_train_1_d2v, y_train1)
y_pred1_svc_d2v = SVC_model.predict(X_test_1_d2v) 

In [52]:
# Logistic Regression
acc_lr_d2v1 = accuracy_score(y_test1, y_pred1_lr_d2v)
print("Logistic Regression")
print("Accuracy:", acc_lr_d2v1)
print(classification_report(y_test1, y_pred1_lr_d2v))

accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train: Reddit / test: Twitter",
        "model": "Logistic Regression",
        "representation": "Doc2Vec",
        "accuracy": acc_lr_d2v1
    }])
], ignore_index=True)

# SVM
acc_svc_d2v1 = accuracy_score(y_test1, y_pred1_svc_d2v)
print("SVM")
print("Accuracy:", acc_svc_d2v1)
print(classification_report(y_test1, y_pred1_svc_d2v))

accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train: Reddit / test: Twitter",
        "model": "SVM",
        "representation": "Doc2Vec",
        "accuracy": acc_svc_d2v1
    }])
], ignore_index=True)

Logistic Regression
Accuracy: 0.6270362960845955
              precision    recall  f1-score   support

           0       0.58      0.73      0.65      4924
           1       0.69      0.54      0.61      5573

    accuracy                           0.63     10497
   macro avg       0.64      0.63      0.63     10497
weighted avg       0.64      0.63      0.62     10497

SVM
Accuracy: 0.6106506620939316
              precision    recall  f1-score   support

           0       0.56      0.80      0.66      4924
           1       0.72      0.44      0.55      5573

    accuracy                           0.61     10497
   macro avg       0.64      0.62      0.60     10497
weighted avg       0.64      0.61      0.60     10497



### Training data: Twitter, testing data: Reddit

In [53]:
log_reg_model_2.fit(X_train_2_d2v, y_train2)
y_pred2_lr_d2v = log_reg_model_2.predict(X_test_2_d2v) 

SVC_model_2.fit(X_train_2_d2v, y_train2)
y_pred2_svc_d2v = SVC_model_2.predict(X_test_2_d2v) 

In [54]:
# Logistic Regression
acc_lr_d2v2 = accuracy_score(y_test2, y_pred2_lr_d2v)
print("Logistic Regression")
print("Accuracy:", acc_lr_d2v2)
print(classification_report(y_test2, y_pred2_lr_d2v))

accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train: Twitter / test: Reddit",
        "model": "Logistic Regression",
        "representation": "Doc2Vec",
        "accuracy": acc_lr_d2v2
    }])
], ignore_index=True)

# SVM
acc_svc_d2v2 = accuracy_score(y_test2, y_pred2_svc_d2v)
print("SVM")
print("Accuracy:", acc_svc_d2v2)
print(classification_report(y_test2, y_pred2_svc_d2v))

accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train: Twitter / test: Reddit",
        "model": "SVM",
        "representation": "Doc2Vec",
        "accuracy": acc_svc_d2v2
    }])
], ignore_index=True)

Logistic Regression
Accuracy: 0.7703126816226897
              precision    recall  f1-score   support

           0       0.71      0.63      0.67      3189
           1       0.80      0.85      0.82      5414

    accuracy                           0.77      8603
   macro avg       0.76      0.74      0.75      8603
weighted avg       0.77      0.77      0.77      8603

SVM
Accuracy: 0.54818086713937
              precision    recall  f1-score   support

           0       0.43      0.62      0.50      3189
           1       0.69      0.51      0.58      5414

    accuracy                           0.55      8603
   macro avg       0.56      0.56      0.54      8603
weighted avg       0.59      0.55      0.56      8603



### Training data: 80%, testing data: 20%

In [55]:
log_reg_model_3.fit(X_train_3_d2v, y_train3)
y_pred3_lr_d2v = log_reg_model_3.predict(X_test_3_d2v) 

SVC_model_3.fit(X_train_3_d2v, y_train3)
y_pred3_svc_d2v = SVC_model_3.predict(X_test_3_d2v) 

In [56]:
# Logistic Regression
acc_lr_d2v3 = accuracy_score(y_test3, y_pred3_lr_d2v)
print("Logistic Regression")
print("Accuracy:", acc_lr_d2v3)
print(classification_report(y_test3, y_pred3_lr_d2v))

accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train/test 80/20 split",
        "model": "Logistic Regression",
        "representation": "Doc2Vec",
        "accuracy": acc_lr_d2v3
    }])
], ignore_index=True)

# SVM
acc_svc_d2v3 = accuracy_score(y_test3, y_pred3_svc_d2v)
print("SVM")
print("Accuracy:", acc_svc_d2v3)
print(classification_report(y_test3, y_pred3_svc_d2v))

accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train/test 80/20 split",
        "model": "SVM",
        "representation": "Doc2Vec",
        "accuracy": acc_svc_d2v3
    }])
], ignore_index=True)

Logistic Regression
Accuracy: 0.768586387434555
              precision    recall  f1-score   support

           0       0.76      0.67      0.71      1630
           1       0.77      0.84      0.81      2190

    accuracy                           0.77      3820
   macro avg       0.77      0.76      0.76      3820
weighted avg       0.77      0.77      0.77      3820

SVM
Accuracy: 0.8068062827225131
              precision    recall  f1-score   support

           0       0.80      0.73      0.76      1630
           1       0.81      0.87      0.84      2190

    accuracy                           0.81      3820
   macro avg       0.81      0.80      0.80      3820
weighted avg       0.81      0.81      0.81      3820



In [57]:
accuracy_df

,dataset_split,model,representation,accuracy
0,train: Reddit / test: Twitter,MultinomialNB,TF-IDF,0.667429
1,train: Twitter / test: Reddit,MultinomialNB,TF-IDF,0.800070
2,train/test 80/20 split,MultinomialNB,TF-IDF,0.810995
3,train: Reddit / test: Twitter,Logistic Regression,TF-IDF,0.737068
4,train: Reddit / test: Twitter,SVM,TF-IDF,0.730494
5,train: Twitter / test: Reddit,Logistic Regression,TF-IDF,0.792747
6,train: Twitter / test: Reddit,SVM,TF-IDF,0.802976
7,train/test 80/20 split,Logistic Regression,TF-IDF,0.866754
8,train/test 80/20 split,SVM,TF-IDF,0.881414
9,train: Reddit / test: Twitter,Logistic Regression,Doc2Vec,0.627036


# Transformer

- https://huggingface.co/docs/transformers/tasks/sequence_classification

In [58]:
import torch
from datasets import Dataset, concatenate_datasets
import evaluate

In [59]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer

https://huggingface.co/distilbert/distilbert-base-uncased

In [60]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

- set max_length = 128 to speed up training

In [61]:
def format_data(dataframe):
    dataframe = dataframe.rename(columns={'full_text': 'text'})
    dataset = Dataset.from_pandas(dataframe)
    return dataset

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128)

In [62]:
# Apply data formatting to train and test sets
datasets = []
for df in df_files_new:
    datasets.append(format_data(df))
    
# Combine datasets and split into train and tests sets as before
full_dataset = concatenate_datasets(datasets)
full_dataset = full_dataset.shuffle(seed=42)
full_dataset = full_dataset.map(preprocess_function, batched=True)

dataset = full_dataset.train_test_split(test_size=0.2)
train_ds = dataset["train"]
test_ds = dataset["test"]

Map:   0%|          | 0/19100 [00:00<?, ? examples/s]

In [63]:
# Check format
train_ds[0]

{'label': 1,
 'text': 'keep on one step at a time. follow happy followme motivation lifestyle inspiration happiness blessed quotes believeinyourself trustyourself follow, happy, followme, motivation, lifestyle, inspiration, happiness, blessed',
 'input_ids': [101,
  2562,
  2006,
  2028,
  3357,
  2012,
  1037,
  2051,
  1012,
  3582,
  3407,
  3582,
  4168,
  14354,
  9580,
  7780,
  8404,
  10190,
  16614,
  2903,
  24300,
  22957,
  2884,
  2546,
  3404,
  29337,
  22573,
  10270,
  3582,
  1010,
  3407,
  1010,
  3582,
  4168,
  1010,
  14354,
  1010,
  9580,
  1010,
  7780,
  1010,
  8404,
  1010,
  10190,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [64]:
# Create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Evaluation

In [65]:
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

### Training

In [66]:
# Define label mappings
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [67]:
# Load pre-trained DistilBERT model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [69]:
# Train the model only if is_train is True

training_args = TrainingArguments(
        output_dir="Transformer",
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.0,
        #evaluation_strategy="no",           
        save_strategy="no",               
        logging_steps=50,                        
        report_to="none",                     
        push_to_hub=False,
        no_cuda=True,
    )

if is_train:
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        processing_class=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

c:\Users\oonas\OneDrive\Työpöytä\DHHB\project\.venv\Lib\site-packages\transformers\training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [177]:
if is_train:
    trainer.model.save_pretrained("Transformer_backup")
    tokenizer.save_pretrained("Transformer_backup")

In [70]:
# Take trained model
model = AutoModelForSequenceClassification.from_pretrained("Transformer_backup")
tokenizer = AutoTokenizer.from_pretrained("Transformer_backup")

new_trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_ds,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [71]:
# Evaluate the model
eval_result = new_trainer.evaluate()
print("Accuracy:", eval_result["eval_accuracy"])


Accuracy: 0.9649214659685864


In [72]:
accuracy_df = pd.concat([
    accuracy_df,
    pd.DataFrame([{
        "dataset_split": "train/test 80/20 split",
        "model": "Transformer",
        "representation": "Tokenizer",
        "accuracy": eval_result["eval_accuracy"]
    }])
], ignore_index=True)

In [73]:
accuracy_df

,dataset_split,model,representation,accuracy
0,train: Reddit / test: Twitter,MultinomialNB,TF-IDF,0.667429
1,train: Twitter / test: Reddit,MultinomialNB,TF-IDF,0.800070
2,train/test 80/20 split,MultinomialNB,TF-IDF,0.810995
3,train: Reddit / test: Twitter,Logistic Regression,TF-IDF,0.737068
4,train: Reddit / test: Twitter,SVM,TF-IDF,0.730494
5,train: Twitter / test: Reddit,Logistic Regression,TF-IDF,0.792747
6,train: Twitter / test: Reddit,SVM,TF-IDF,0.802976
7,train/test 80/20 split,Logistic Regression,TF-IDF,0.866754
8,train/test 80/20 split,SVM,TF-IDF,0.881414
9,train: Reddit / test: Twitter,Logistic Regression,Doc2Vec,0.627036


In [74]:
# Save accuracies to CSV
accuracy_df.to_csv('accuracies.csv', index=False)